In [4]:
# CNN Feature -> Transformer Decoder for MedicalCaptionGroup

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


In [5]:
# Example Vocabulary Size (Adjust based on your tokenizer)
vocab_size = 5000

# Parameters
feature_dim = 2048  # ResNet50 output size
embed_dim = 512
num_heads = 8
hidden_dim = 2048
num_layers = 4
max_caption_length = 20
start_token_idx = 1
end_token_idx = 2

In [6]:
# Custom Dataset
class MedicalCaptionDataset(Dataset):
    def __init__(self, features, captions):
        self.features = features
        self.captions = captions

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        caption = torch.tensor(self.captions[idx], dtype=torch.long)
        return feature, caption

In [7]:
# Transformer Decoder Model
class TransformerDecoder(nn.Module):
    def __init__(self, feature_dim, vocab_size, embed_dim, num_heads, hidden_dim, num_layers):
        super(TransformerDecoder, self).__init__()
        self.feature_embed = nn.Linear(feature_dim, embed_dim)
        self.caption_embed = nn.Embedding(vocab_size, embed_dim)

        decoder_layer = nn.TransformerDecoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)

        self.fc_out = nn.Linear(embed_dim, vocab_size)

    def forward(self, features, captions):
        features = self.feature_embed(features).unsqueeze(0)
        captions_embed = self.caption_embed(captions).permute(1, 0, 2)

        output = self.transformer_decoder(tgt=captions_embed, memory=features)
        output = self.fc_out(output)
        return output

In [8]:
dataset = MedicalCaptionDataset(all_features, all_captions)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

NameError: name 'all_features' is not defined